Zadanie 1

In [4]:
import numpy as np
import pandas as pd
import pyreadr

# 1.1 Wczytanie danych
crypto = pyreadr.read_r('crypto.rds')
df = crypto[None] # DataFrame z R

df.head()

# 1.2 Dane dotyczące Bitcoina
df = df[df['Currency'].str.contains('bitcoin', case=False, na=False)]

# 1.3 Pozostawienie kolumn 'Date' i 'Close'
df = df[['Date', 'Close']]

# 1.4 Zmiana typu kolumny 'Date' na typ date
df['Date'] = pd.to_datetime(df['Date'])

# 1.5 Dodatkowa kolumna 'Rate'
df['Rate'] = (df['Close'] - df['Close'].shift(1)) / df['Close'].shift(1)

# 1.6 Sortowanie Rate malejąco
df = df.sort_values(by='Rate', ascending=False)
df.head()

,Date,Close,Rate
6478,2018-02-11,8129.97,15.255714
6477,2017-10-23,500.13,2.499370
8550,2017-03-27,26.82,1.449315
8549,2017-03-28,10.95,0.596210
6354,2017-08-03,364.05,0.562111


Zadanie 2

In [8]:
df_albums = pd.read_csv('albums.csv')
df_albums.head()

# liczba wierszy i kolumn
df_albums.shape

# struktura danych
df_albums.info()

# sprawdzenie braków danych (NA)
df_albums.isna().sum()

# liczba albumów na artystę
albums_per_artist = (
    df_albums
    .groupby('artist_id')
    .size()
    .reset_index(name='n_albums')
    .sort_values(by='n_albums', ascending=False)
)

albums_per_artist

# liczba albumów w każdym gatunku
albums_per_genre = (
    df_albums
    .groupby('genre')
    .size()
    .reset_index(name='n_albums')
    .sort_values(by='n_albums', ascending=False)
)

albums_per_genre

# statystyki ocen MTV
mtv_stats = df_albums['mtv_critic'].agg(
    mean_mtv_critic='mean',
    median_mtv_critic='median',
    min_mtv_critic='min',
    max_mtv_critic='max'
)

mtv_stats

# statystyki ocen Music Maniac
music_maniac_stats = df_albums['music_maniac_critic'].agg(
    mean_music_maniac_critic='mean',
    median_music_maniac_critic='median',
    min_music_maniac_critic='min',
    max_music_maniac_critic='max'
)

music_maniac_stats




<class 'pandas.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    100000 non-null  int64  
 1   artist_id             100000 non-null  int64  
 2   album_title           100000 non-null  str    
 3   genre                 100000 non-null  str    
 4   year_of_pub           100000 non-null  int64  
 5   num_of_tracks         100000 non-null  int64  
 6   num_of_sales          100000 non-null  int64  
 7   rolling_stone_critic  100000 non-null  float64
 8   mtv_critic            100000 non-null  float64
 9   music_maniac_critic   100000 non-null  float64
dtypes: float64(3), int64(5), str(2)
memory usage: 7.6 MB


mean_music_maniac_critic      2.748225
median_music_maniac_critic    3.000000
min_music_maniac_critic       0.500000
max_music_maniac_critic       5.000000
Name: music_maniac_critic, dtype: float64

Zadanie 3

In [ ]:
suicides = pyreadr.read_r('suicides.rds')
df_suicides = suicides[None]

# 3.1 średnia liczba samobójstw na 100k mieszkańców (1985–2016)
suicides_summary = (
    df_suicides.groupby('country', as_index=False)
    .agg(avg_suicides_per_100k=('suicides.100k.pop', 'mean'))
)

# 5 krajów z największą liczbą samobójstw na 100k mieszkańców
top5 = (suicides_summary.sort_values(by="avg_suicides_per_100k", ascending=False).head(5))

# 5 krajów z najmniejszą liczbą samobójstw na 100k mieszkańców
bottom5 = (suicides_summary.sort_values(by="avg_suicides_per_100k", ascending=True).head(5))

# Wyniki
print(f"5 krajów (najwięcej samobójstw na 100k):\n{top5}")
print(f"\n5 krajów (najmniej samobójstw na 100k):\n{bottom5}")

# 3.2 Łączna liczba samobójstw na 100k dla każdego roku
suicides_by_year = (
    df_suicides
    # .query("year == 2012")  # opcjonalnie – do sprawdzenia jednego roku
    .groupby("year", as_index=False)
    .agg(
        total_suicides=("suicides_no", "sum"),
        total_population=("population", "sum")
    )
)

# Obliczenie wskaźnika na 100k mieszkańców
suicides_by_year["suicides_per_100k"] = (
    suicides_by_year["total_suicides"]
    / suicides_by_year["total_population"]
    * 100_000
)

# Wynik
print(suicides_by_year)

5 krajów (najwięcej samobójstw na 100k):
               country  avg_suicides_per_100k
52           Lithuania              40.415573
87           Sri Lanka              35.295152
75  Russian Federation              34.892377
40             Hungary              32.761516
11             Belarus              31.075913

5 krajów (najmniej samobójstw na 100k):
                  country  avg_suicides_per_100k
76  Saint Kitts and Nevis               0.000000
27               Dominica               0.000000
45                Jamaica               0.521765
1     Antigua and Barbuda               0.552901
65                   Oman               0.736111
    year  total_suicides  total_population  suicides_per_100k
0   1985          116063        1008600086          11.507336
1   1986          120670        1029909613          11.716562
2   1987          126842        1095029726          11.583430
3   1988          121026        1054094424          11.481514
4   1989          160244        122551